In [1]:
import sys as sys
import pandas as pd
import urllib.request
from urllib.request import urlopen
import io
import requests
from PyPDF2 import PdfFileReader
#from tika import parser
#from bs4 import BeautifulSoup

import nltk
from nltk import pos_tag, word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import wordnet, WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet, stopwords

In [2]:
pd.set_option('display.max_colwidth', None) # PD has a limit of 50 characters.  This takes out the limit and uses the full text.

In [3]:
NRC_ASP_df = pd.read_csv('./input_data/NRC_ASP_DATA_from_Public_ASP_Dashboard_wLER.csv')

In [4]:
NRC_ASP_df.head(5)

,Plant,Event Date,LER / IR,Description,Result,ASP Analysis
0,D.C. Cook 2,9/4/2020,https://www.nrc.gov/docs/ML2031/ML20311A129.pdf,Manual reactor trip and automatic SI due to failed open pressurizer spray valve,0.000010,https://www.nrc.gov/docs/ML2103/ML21035A236.pdf
1,Duane Arnold,8/10/2020,https://www.nrc.gov/docs/ML2028/ML20283A373.pdf,LOOP caused by high winds during derecho,0.000800,https://www.nrc.gov/docs/ML2105/ML21056A382.pdf
2,Brunswick 1,8/3/2020,https://www.nrc.gov/docs/ML2026/ML20265A162.pdf,LOOP during Hurricane Isaias,0.000020,https://www.nrc.gov/docs/ML2029/ML20294A552.pdf
3,Fitzpatrick,4/10/2020,https://www.nrc.gov/docs/ML2016/ML20161A405.pdf,High pressure coolant injection inoperable due to oil leak,0.000003,https://www.nrc.gov/docs/ML2110/ML21105A543.pdf
4,Quad Cities 2,3/30/2020,https://www.nrc.gov/docs/ML2014/ML20149K600.pdf,Electromatic relief valve 3D failed to actuate due to out-of-specification plunger,0.000030,https://www.nrc.gov/docs/ML2102/ML21029A319.pdf


In [6]:
NRC_ASP_df['ASP Analysis'].isnull().sum()

0

In [8]:
NRC_ASP_df['LER / IR'].isnull().sum()

668

In [9]:
URL = 'https://www.nrc.gov/docs/ML2105/ML21055A029.pdf' # https://wellsr.com/python/read-pdf-files-with-python-using-pypdf2/, geeksforgeeks.org/working-with-pdf-files-in-python/
req = urllib.request.Request(URL)
remote_file = urllib.request.urlopen(req).read()
remote_file_bytes = io.BytesIO(remote_file)
pdfdoc_remote = PdfFileReader(remote_file_bytes)

In [10]:
pdfdoc_remote.numPages

8

In [11]:
all_text = str('')
for i in range(pdfdoc_remote.numPages): # iterates thru all the pages.
    current_page = pdfdoc_remote.getPage(i)
    #print("===================") # Prints "===" between content on page.
    #print("Content on page:" + str(i + 1))
    #print("===================")
    #print(current_page.extractText())
    all_text += current_page.extractText()

In [12]:
all_text

' \n \n1\n \nFinal\n \nASP Analysis\n \n\n \nPrecursor\n \nAccident Sequence Precursor Program \n\n \nOffice of Nuclear Regulatory Research\n \nNorth Anna Power \nStation, Unit 1\n \nDegraded Upper Cylinder Piston Pin Bushing Discovered during \nMaintenance Activities on Emergency Diesel \nGenerator\n \nEvent Date:\n \n2/18/2020\n \nLER:\n \n338\n-\n20\n-\n001\n \n\nCDP =\n \n5\n×\n10\n-\n6\n \nIR:\n \n05000338/2020003\n \nPlant Type:\n \nWestinghouse Three\n-\nLoop Pressurized Water Reactor (PWR) with a Large, Dry \nContainment\n \nPlant Operating Mode\n \n(Reactor Power Level):\n \nMode 1 (100% \nReactor Power)\n \nAnalyst:\n \nGary Wang\n \nReviewer:\n \nChris Hunter\n \nCompletion Date\n:\n \n3\n/\n2\n/\n2021\n \n1\n \nE\nXE\nCUTIVE SUMMARY\n \n \n \nOn February 18, 2020, \nthe licensee discovered \nbrass shavings \ni\nn the upper crankcase of \ne\nmergency \nd\niesel \ng\nenerator (EDG)\n \n\n \nduring planned maintenance activities.\n \n \nF\nurther \ninvestigation \nrevealed deg

In [13]:
all_text = all_text.replace('\n', ' ') # Need to clean this to remove the "\n" and use the text normalization function.  
all_text = all_text.replace('_', ' ')
all_text
# Can't remove 2 letter words as for example TS is an accronym used accross the corpus.
# Can then add this as a column in the dataframe.

'    1   Final   ASP Analysis      Precursor   Accident Sequence Precursor Program     Office of Nuclear Regulatory Research   North Anna Power  Station, Unit 1   Degraded Upper Cylinder Piston Pin Bushing Discovered during  Maintenance Activities on Emergency Diesel  Generator   Event Date:   2/18/2020   LER:   338 - 20 - 001    CDP =   5 × 10 - 6   IR:   05000338/2020003   Plant Type:   Westinghouse Three - Loop Pressurized Water Reactor (PWR) with a Large, Dry  Containment   Plant Operating Mode   (Reactor Power Level):   Mode 1 (100%  Reactor Power)   Analyst:   Gary Wang   Reviewer:   Chris Hunter   Completion Date :   3 / 2 / 2021   1   E XE CUTIVE SUMMARY       On February 18, 2020,  the licensee discovered  brass shavings  i n the upper crankcase of  e mergency  d iesel  g enerator (EDG)      during planned maintenance activities.     F urther  investigation  revealed degradation of the number one upper piston pin bushing.     The apparent  cause was determined to be a degradat

In [14]:
def url_is_alive(url): # https://gist.github.com/dehowell/884204
    """
    Checks that a given URL is reachable.
    :param url: A URL
    :rtype: bool
    """
    request = urllib.request.Request(url)
    request.get_method = lambda: 'HEAD'

    try:
        urllib.request.urlopen(request)
        return True
    except urllib.request.HTTPError:
        return False

In [17]:
def extract_link_text(target_link_column, column_title):
    for row in range(NRC_ASP_df.shape[0]): # Column with url
        URL = NRC_ASP_df.at[row, target_link_column]
        if str('nrc.gov') in str(URL):
            if url_is_alive(URL) == True:
                req = urllib.request.Request(URL)
                remote_file = urllib.request.urlopen(req).read()
                remote_file_bytes = io.BytesIO(remote_file)
                pdfdoc_remote = PdfFileReader(remote_file_bytes)
                all_text = str('')
                for i in range(pdfdoc_remote.numPages): # iterates thru all the pages.
                    current_page = pdfdoc_remote.getPage(i)
                    all_text += current_page.extractText()
                NRC_ASP_df.at[row, column_title+'_RPT_Text'] = all_text
            elif url_is_alive(URL) == False:
                NRC_ASP_df.at[row, column_title+'_RPT_Text'] = str('REPORT UNAVAILABLE OR URL DOES NOT EXIST')
        else:
            NRC_ASP_df.at[row, column_title+'_RPT_Text'] = str('REPORT UNAVAILABLE OR URL DOES NOT EXIST')

In [18]:
extract_link_text('ASP Analysis', 'ASP')

In [19]:
extract_link_text('LER / IR', 'LER')

In [21]:
NRC_ASP_df.head(10)

Plant  Event Date  \
0     D.C. Cook 2    9/4/2020   
1    Duane Arnold   8/10/2020   
2     Brunswick 1    8/3/2020   
3     Fitzpatrick   4/10/2020   
4   Quad Cities 2   3/30/2020   
5    North Anna 1   2/18/2020   
6         Surry 2  11/20/2019   
7  Browns Ferry 3    3/9/2019   
8  Peach Bottom 2   6/13/2018   
9         Pilgrim    1/8/2019   

                                          LER / IR  \
0  https://www.nrc.gov/docs/ML2031/ML20311A129.pdf   
1  https://www.nrc.gov/docs/ML2028/ML20283A373.pdf   
2  https://www.nrc.gov/docs/ML2026/ML20265A162.pdf   
3  https://www.nrc.gov/docs/ML2016/ML20161A405.pdf   
4  https://www.nrc.gov/docs/ML2014/ML20149K600.pdf   
5  https://www.nrc.gov/docs/ML2012/ML20122A056.pdf   
6  https://www.nrc.gov/docs/ML2002/ML20028D616.pdf   
7  https://www.nrc.gov/docs/ML1918/ML19189A125.pdf   
8  https://www.nrc.gov/docs/ML1906/ML19064B342.pdf   
9  https://www.nrc.gov/docs/ML1906/ML19064A593.pdf   

                                                                                  Description  \
0             Manual reactor trip and automatic SI due to failed open pressurizer spray valve   
1                                                    LOOP caused by high winds during derecho   
2                                                                LOOP during Hurricane Isaias   
3                                  High pressure coolant injection inoperable due to oil leak   
4          Electromatic relief valve 3D failed to actuate due to out-of-specification plunger   
5  Degraded upper cylinder piston pin bushing discovered during maintenance activities on EDG   
6                          AFW system flow diversion due to failed pump discharge check valve   
7                                                               LOOP caused by operator error   
8                                                           EDG air inlet check valve failure   
9                                               RCIC system fails during surveillance testing   

     Result                                     ASP Analysis  \
0  0.000010  https://www.nrc.gov/docs/ML2103/ML21035A236.pdf   
1  0.000800  https://www.nrc.gov/docs/ML2105/ML21056A382.pdf   
2  0.000020  https://www.nrc.gov/docs/ML2029/ML20294A552.pdf   
3  0.000003  https://www.nrc.gov/docs/ML2110/ML21105A543.pdf   
4  0.000030  https://www.nrc.gov/docs/ML2102/ML21029A319.pdf   
5  0.000002  https://www.nrc.gov/docs/ML2105/ML21055A029.pdf   
6  0.000005  https://www.nrc.gov/docs/ML2021/ML20212L517.pdf   
7  0.000003  https://www.nrc.gov/docs/ML1928/ML19288A331.pdf   
8  0.000005  https://www.nrc.gov/docs/ML1834/ML18341A206.pdf   
9  0.000003  https://www.nrc.gov/docs/ML1916/ML19169A279.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [22]:
NRC_ASP_df.to_csv('./output_data/NRC_ASP_DATA_from_Public_ASP_Dashboard_wRawText.csv', index = False, header = True)
# Note that MS Excel does not show correctly because the Text is too long.

In [23]:
row = 5

In [24]:
NRC_ASP_df.at[row, 'Plant']

'North Anna 1'

In [25]:
NRC_ASP_df.at[row, 'ASP Analysis']

'https://www.nrc.gov/docs/ML2105/ML21055A029.pdf'

In [26]:
NRC_ASP_df.at[row, 'ASP_RPT_Text']

' \n \n1\n \nFinal\n \nASP Analysis\n \n\n \nPrecursor\n \nAccident Sequence Precursor Program \n\n \nOffice of Nuclear Regulatory Research\n \nNorth Anna Power \nStation, Unit 1\n \nDegraded Upper Cylinder Piston Pin Bushing Discovered during \nMaintenance Activities on Emergency Diesel \nGenerator\n \nEvent Date:\n \n2/18/2020\n \nLER:\n \n338\n-\n20\n-\n001\n \n\nCDP =\n \n5\n×\n10\n-\n6\n \nIR:\n \n05000338/2020003\n \nPlant Type:\n \nWestinghouse Three\n-\nLoop Pressurized Water Reactor (PWR) with a Large, Dry \nContainment\n \nPlant Operating Mode\n \n(Reactor Power Level):\n \nMode 1 (100% \nReactor Power)\n \nAnalyst:\n \nGary Wang\n \nReviewer:\n \nChris Hunter\n \nCompletion Date\n:\n \n3\n/\n2\n/\n2021\n \n1\n \nE\nXE\nCUTIVE SUMMARY\n \n \n \nOn February 18, 2020, \nthe licensee discovered \nbrass shavings \ni\nn the upper crankcase of \ne\nmergency \nd\niesel \ng\nenerator (EDG)\n \n\n \nduring planned maintenance activities.\n \n \nF\nurther \ninvestigation \nrevealed deg

In [28]:
NRC_ASP_df.at[row, 'LER / IR']

'https://www.nrc.gov/docs/ML2012/ML20122A056.pdf'

In [27]:
NRC_ASP_df.at[row, 'LER_RPT_Text']

'10CFR50.73 Virginia Electric and Power Company North Anna Power Station 1022 Haley Drive Mineral, Virginia 23117 April 21, 2020 Attention: Document Control Desk U. S. Nuclear Regulatory Commission Washington, DC 20555-0001 Dear Sir or Madam: Serial No.: 20-141 NAPS: CVNV Docket Nos.: 50-338 License Nos.: NPF-4 Pursuant to 10CFR50.73, Virginia Electric and Power Company hereby submits the following Licensee Event Report applicable to North Anna Power Station Unit 1. This revision corrects an error in the original submission for the equipment\'s return to service date. Report No. 50-338/2020-001-01 This report has been reviewed by the Facility Safety Review Committee and will be forwarded to the Management Safety Review Committee for its review. Sincerely, f"J.P~& N. Larry Lane Site Vice President North Anna Power Station Enclosure Commitments contained in this letter: None cc: United States Nuclear Regulatory Commission Region II Marquis One Tower 245 Peachtree Center Ave., NE, Suite 1

# Text Data Cleaning

In [29]:
NRC_ASP_df = pd.read_csv('./output_data/NRC_ASP_DATA_from_Public_ASP_Dashboard_wRawText.csv')
# Note that MS Excel does not show correctly because the Text is too long.

In [30]:
NRC_ASP_df.head(3)

Plant Event Date                                         LER / IR  \
0   D.C. Cook 2   9/4/2020  https://www.nrc.gov/docs/ML2031/ML20311A129.pdf   
1  Duane Arnold  8/10/2020  https://www.nrc.gov/docs/ML2028/ML20283A373.pdf   
2   Brunswick 1   8/3/2020  https://www.nrc.gov/docs/ML2026/ML20265A162.pdf   

                                                                       Description  \
0  Manual reactor trip and automatic SI due to failed open pressurizer spray valve   
1                                         LOOP caused by high winds during derecho   
2                                                     LOOP during Hurricane Isaias   

    Result                                     ASP Analysis  \
0  0.00001  https://www.nrc.gov/docs/ML2103/ML21035A236.pdf   
1  0.00080  https://www.nrc.gov/docs/ML2105/ML21056A382.pdf   
2  0.00002  https://www.nrc.gov/docs/ML2029/ML20294A552.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [31]:
# Stopwords used in this DATASET.
stopwords_to_add = [] #List, for example:  ['employee','employees']
remove_as_stopword = ['no']
#stopwords.words('english') # Default Stopwords from NLTK
stopwords_english = list(filter(lambda w: w not in remove_as_stopword, stopwords.words('english')))
stopwords_custom = stopwords_english + stopwords_to_add
print(stopwords_custom)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 

In [32]:
def text_normalization(narrative):
    lemmatizer = WordNetLemmatizer()
    #stemmer = PorterStemmer() # Stemming also could make the word unreadable.
    narrative = str(narrative)
    df = pd.DataFrame({'': [narrative]})
    df['']=df[''].str.replace('\n', ' ') # Remove the "\n" and replace with space.  
    df['']=df[''].str.replace('_', ' ') # Remove the "_" and replace with space.  
    # Covert narrative to lower case.
    df['']=df[''].str.lower()
    # Remove numbers
    df['']=df[''].str.replace("\d+", " ")
    # Remove special characters
    df['']=df[''].str.replace("[^\w\s]", " ")
    # Replace multiple spaces with single
    df['']=df[''].str.replace('\s+', ' ', regex=True)
    narrative=str(df[0:1])
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(narrative) 
    filtered_words = [w for w in tokens if len(w) > 1 if not w in stopwords_custom]
    lemma_words=[lemmatizer.lemmatize(w) for w in filtered_words]
    return " ".join(lemma_words)

In [33]:
def text_normalization_wstemming(narrative):
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer() # Stemming also could make the word unreadable.
    narrative = str(narrative)
    df = pd.DataFrame({'': [narrative]})
    # Covert narrative to lower case.
    df['']=df[''].str.lower()
    # Remove numbers
    df['']=df[''].str.replace("\d+", " ")
    # Remove special characters
    df['']=df[''].str.replace("[^\w\s]", " ")
    # Replace multiple spaces with single
    df['']=df[''].str.replace('\s+', ' ', regex=True)
    narrative=str(df[0:1])
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(narrative) 
    filtered_words = [w for w in tokens if len(w) > 1 if not w in stopwords_custom] # Note for the NRC data it was reduced to 1 to make sure do not get rid of 2-letter accronyms (e.g., TS)
    lemma_words=[lemmatizer.lemmatize(w) for w in filtered_words]
    stem_words = [stemmer.stem(w) for w in lemma_words]
    return " ".join(stem_words)

In [34]:
NRC_ASP_df['ASP_RPT_norm_text'] = NRC_ASP_df['ASP_RPT_Text'].apply(text_normalization)

In [35]:
NRC_ASP_df['ASP_RPT_norm_text_wstem'] = NRC_ASP_df['ASP_RPT_Text'].apply(text_normalization_wstemming)
# Found that stemmed text works best for search querying.

In [36]:
NRC_ASP_df['LER_RPT_norm_text'] = NRC_ASP_df['LER_RPT_Text'].apply(text_normalization)

In [37]:
NRC_ASP_df['LER_RPT_norm_text_wstem'] = NRC_ASP_df['LER_RPT_Text'].apply(text_normalization_wstemming)
# Found that stemmed text works best for search querying.

In [38]:
NRC_ASP_df.head(1)

Plant Event Date                                         LER / IR  \
0  D.C. Cook 2   9/4/2020  https://www.nrc.gov/docs/ML2031/ML20311A129.pdf   

                                                                       Description  \
0  Manual reactor trip and automatic SI due to failed open pressurizer spray valve   

    Result                                     ASP Analysis  \
0  0.00001  https://www.nrc.gov/docs/ML2103/ML21035A236.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [39]:
# Check any cell.
row = 1
NRC_ASP_df.at[row, 'Plant']

'Duane Arnold'

In [40]:
NRC_ASP_df.to_csv('./output_data/NRC_ASP_DATA_from_Public_ASP_Dashboard_wNormText.csv', index = False, header = True)
# Note that MS Excel does not show correctly because the Text is too long.